In [17]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

In [18]:
# Load the data
df = pd.read_csv('labeled-data.csv')

# Extract features and labels
# Only include features F3,F4,C3,Cz,C4,P3,P4
X = df.iloc[:, 2:9].values 
y = df['label'].values

In [19]:
# Standardize the data
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Specify the order of your classes
class_order = ['N', 'L', 'R', 'Done']

label_encoder = LabelEncoder()
label_encoder.fit(class_order)

y_encoded = label_encoder.fit_transform(df['label'])

# Convert labels to categorical
y_one_hot = to_categorical(y_encoded)

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y_one_hot, test_size=0.1, random_state=42)


In [20]:
# Reshape the data for Conv1D layer
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

In [21]:
# Build the CNN model
# 1 CNN & 1 Non-Dense layer
model = Sequential()
model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(10, activation='relu'))
model.add(Dense(4, activation='relu'))
print("Shape of y_train:", y_train.shape)


Shape of y_train: (1383, 4)


In [22]:
model.add(Dense(y_train.shape[1], activation='softmax'))

In [23]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.1)


Epoch 1/10
39/39 [==============================] - 0s 4ms/step - loss: 1.3734 - accuracy: 0.3601 - val_loss: 1.3647 - val_accuracy: 0.4245
Epoch 2/10
39/39 [==============================] - 0s 1ms/step - loss: 1.3498 - accuracy: 0.4116 - val_loss: 1.3410 - val_accuracy: 0.4820
Epoch 3/10
39/39 [==============================] - 0s 1ms/step - loss: 1.3257 - accuracy: 0.4325 - val_loss: 1.3147 - val_accuracy: 0.4820
Epoch 4/10
39/39 [==============================] - 0s 1ms/step - loss: 1.2959 - accuracy: 0.4325 - val_loss: 1.2734 - val_accuracy: 0.4820
Epoch 5/10
39/39 [==============================] - 0s 1ms/step - loss: 1.2484 - accuracy: 0.4333 - val_loss: 1.2050 - val_accuracy: 0.4820
Epoch 6/10
39/39 [==============================] - 0s 1ms/step - loss: 1.1932 - accuracy: 0.4341 - val_loss: 1.1547 - val_accuracy: 0.4820
Epoch 7/10
39/39 [==============================] - 0s 1ms/step - loss: 1.1533 - accuracy: 0.4341 - val_loss: 1.1328 - val_accuracy: 0.4820
Epoch 8/10
39/39 [==

In [24]:
# Evaluate the model on the test set
y_pred = model.predict(X_test)
y_pred_classes = y_pred.argmax(axis=-1)
y_test_classes = y_test.argmax(axis=-1)

accuracy = accuracy_score(y_test_classes, y_pred_classes)
print(f'Test Accuracy: {accuracy}')

5/5 [==============================] - 0s 796us/step
Test Accuracy: 0.44805194805194803
